# **Data-level Parallelism Integrating Project - MOVEMENT RECOGNITION IN SIMT**
Submitted by:
- Cai, Edison
- Dequico, Beverly Joyce
- La’O, Erin Denise
- Relucio, Jan Jhezaree

In [14]:
%%writefile group5_simt.c

#include <stdio.h>
#include <stdlib.h>
#include <ctype.h>
#include <math.h>

// ------- R SQUARE -------
/**float calculateRSquare(int n, float *predict_y, float *actual_y) {
    float sum_x = 0, sum_y = 0, sum_xy = 0, sum_x2 = 0, sum_y2 = 0;

    // Compute the necessary sums
    for (int i = 0; i < n; i++) {
        sum_x += predict_y[i];
        sum_y += actual_y[i];
        sum_xy += predict_y[i] * actual_y[i];
        sum_x2 += predict_y[i] * predict_y[i];
        sum_y2 += actual_y[i] * actual_y[i];
    }

    // Compute the numerator and denominator of the R^2 formula
    float numerator = (n * sum_xy) - (sum_x * sum_y);
    float denominator = sqrtf((n * sum_x2 - sum_x * sum_x) * (n * sum_y2 - sum_y * sum_y));

    if (denominator != 0) {
        return powf(numerator / denominator, 2);
    } else {
        return 0;  // Avoid division by zero
    }
}**/

// ------- GET COORDINATES -------
int checkCoordinates(char inputArr[], double x[], double y[], double z[]) {
    int open = 0;
    char current[10001];
    int index = 0;
    int count = 0;
    int doneX = 0;

    for (int i = 1; inputArr[i] != '\0'; i++) {
        if (inputArr[i] == '[') {
            open++;
        }

        else if ((inputArr[i] == '-') || (isdigit(inputArr[i]) || (inputArr[i] == '.'))) {
            current[index] = inputArr[i];
            index++;
        }

        else if ((inputArr[i] == ',') && (isdigit(inputArr[i+1]))) {
            if (doneX == 0) {
                x[count] = atof(current);
                doneX = 1;
            }
            else if (doneX == 1) {
                y[count] = atof(current);
            }

            //clear
            for (int w = 0; current[w] != '\0'; w++) {
                current[w] = '\0';
                index = 0;
            }
        }

        else if ((inputArr[i] == ']') && (inputArr[i+1] != ']')) {
            z[count] = atof(current);
            count++;
            doneX = 0;

            //clear
            for (int w = 0; current[w] != '\0'; w++) {
                current[w] = '\0';
                index = 0;
            }
        }
    }

    return (open);
}

// ------- ERROR CHECKING -------
int checkError(char inputArr[]) {
    int letterCount = 0;
    int open = 0;
    int close = 0;
    int errorCount = 0;

    // go through array
    for (int i = 0; inputArr[i] != '\0'; i++) {
        // check if array is an array of arrays
        if (inputArr[i+1] == '\0') {
            if ((inputArr[0] != '[') || (inputArr[1] != '[') || (inputArr[i-2] != ']') || (inputArr[i-1] != ']')) {
                printf("Input is not an array of arrays.\n");
                errorCount++;
            }
        }

      // check if there are letters in the input
        if ((inputArr[i] >= 'a' && inputArr[i] <= 'z') || (inputArr[i] >= 'A' && inputArr[i] <= 'Z')) {
            letterCount += 1; // invalid
        }

      // check for balanced brackets
      if (inputArr[i] == '[')
          open++;
      if (inputArr[i] == ']')
          close++;
    }

    if (letterCount > 0) {
        printf("No letters should be in the input.\n");
        errorCount++;
    }
    if (open != close) {
        printf("Brackets not balanced.\n");
        errorCount++;
    }

    if (errorCount > 0)
        return 0;
    if (errorCount == 0)
        return 1;
}

// ------- MAIN PROGRAM -------
int main() {
    char inputArr[10001];
    double x[10001];
    double y[10001];
    double z[10001];
    //char currentChar;
    int numSets = 0;
    int sets = 0;
    int index = 0;
    int valid = 0;
    //int invalidCount = 0;

    while (valid == 0) {
        printf("Input: ");
        fgets(inputArr, sizeof(inputArr), stdin);  // read string

      valid = checkError(inputArr);
    }

    // group arrays
    sets = checkCoordinates(inputArr,x,y,z);
    valid = 0;

    while (valid == 0) {
      printf("\nNumber of sets to group: ");
      scanf("%d", &numSets);

      // check if numSets <= number of sets in input array
      if (numSets > sets)
          printf("Please input valid number of sets.");
      else
          valid = 1;
    }

    // SOLUTION

    // Results
    printf("\nRESULTS:\n");

    return 0;
}

Overwriting group5_simt.c


In [15]:
%%shell
gcc group5_simt.c -o group5_simt
./group5_simt

Input: [[1,2,3],[4,5,6]]

Number of sets to group: 1

RESULTS:


# **CUDA - No Prefetching**

In [ ]:
%%writefile CUDA_movement.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

__global__
void rSquare(size_t n, float *r_square, float *predict_y, float *actual_y) {
    extern __shared__ float shared_mem[];
    float *sum_x = &shared_mem[0];
    float *sum_y = &shared_mem[1];
    float *sum_xy = &shared_mem[2];
    float *sum_x2 = &shared_mem[3];
    float *sum_y2 = &shared_mem[4];

    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    // INITIALIZE SHARED MEMORY VALUES
    if (threadIdx.x == 0) {
        *sum_x = 0;
        *sum_y = 0;
        *sum_xy = 0;
        *sum_x2 = 0;
        *sum_y2 = 0;
    }

    __syncthreads();

    // CALCULATE SUMS IN PARALLEL
    for (int i = index; i < n; i += stride) {
        atomicAdd(sum_x, predict_y[i]);
        atomicAdd(sum_y, actual_y[i]);
        atomicAdd(sum_xy, predict_y[i] * actual_y[i]);
        atomicAdd(sum_x2, predict_y[i] * predict_y[i]);
        atomicAdd(sum_y2, actual_y[i] * actual_y[i]);
    }

    __syncthreads();

    // COMPUTE THE RESULT IN THE FIRST THREAD OF EACH BLOCK
    if (threadIdx.x == 0) {
        float numerator = (n * (*sum_xy)) - (*sum_x * *sum_y);
        float denominator = sqrtf((n * (*sum_x2) - (*sum_x) * (*sum_x)) *
                                   (n * (*sum_y2) - (*sum_y) * (*sum_y)));

        if (denominator != 0) {
            *r_square = powf(numerator / denominator, 2);
        } else {
            *r_square = 0;
        }
    }
}

int main() {
    const size_t ARRAY_SIZE = 60;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    const size_t loope = 1;

    float x[] = {
        -1.752874, -2.256292, -1.909765, -1.609097, -1.612630, -1.596325,
        -1.060799, -0.992284, -1.206335, -0.992564, -1.067933, -1.184888,
        -0.643832, -0.230520, -0.205881, -0.238917, -0.178148, 0.045699,
        0.615788, 1.226356, 1.724358, 2.059628, 2.269062, 2.462253,
        2.639274, 3.064922, 3.660523, 2.973241, 1.993106, 1.778791,
        1.187832, 0.772967, 0.719589, 0.528514, 0.151956, -0.252073,
        -0.767705, -1.313824, -1.724808, -1.277459, -0.656633, 0.653198,
        0.524446, 0.212403, -0.369462, -1.112287, 1.145783, 1.582146,
        -0.294279, -1.176760
    };

    float y[] = {
        2.553555, 0.723740, 0.425348, 0.315908, -0.426752, -0.525212,
        0.275625, 1.406080, 1.259696, 0.672787, 0.647924, 0.882002,
        0.194108, 0.429657, 0.446855, -0.127734, -0.471531, -0.397046,
        -1.648964, -3.096304, -3.641368, -2.870210, -1.495181, -1.100132,
        -0.512399, -0.046665, -0.169440, -1.593415, -2.455789, -2.381501,
        -2.776803, -2.482388, -2.040230, -1.622057, -0.899546, -0.193568,
        0.666911, 1.669011, 1.952349, 0.848186, 0.509708, -0.150707,
        0.364522, -0.391019, -0.436157, 2.044329, -0.160673, 0.363966,
        1.417464, 2.934732
    };

    float z[] = {
        0.768259, 0.323775, 0.320185, 0.255127, 0.298783, 0.132381,
        -0.002706, -0.111566, -0.347882, -0.547531, -0.658053, -0.752105,
        -0.757058, -0.613467, -0.141163, -0.011403, 0.130467, 0.489790,
        0.582970, 0.494018, 0.239686, 0.241039, 0.120031, 0.260811,
        -0.083741, -1.036355, -1.218976, -0.698524, 0.025331, 0.547167,
        1.079794, 1.339301, 0.836297, 0.323817, 0.320261, 0.478903,
        0.459244, 0.480851, 0.576962, 0.590047, 0.514701, 0.164705,
        0.367176, 0.436467, 0.241249, -0.310843, -0.965225, -0.675497,
        -0.632270, -0.745064
    };

    int group_size;
    printf("Enter the number of sets to group together: ");
    scanf("%d", &group_size);

    if (group_size <= 0 || group_size * 2 > ARRAY_SIZE) {
        fprintf(stderr, "Invalid group size.\n");
        return -1;
    }

    const size_t num_elements = ARRAY_SIZE / 2;

    float *actual_y, *predict_y, *r_square;
    cudaMallocManaged(&actual_y, num_elements * sizeof(float));
    cudaMallocManaged(&predict_y, num_elements * sizeof(float));
    cudaMallocManaged(&r_square, sizeof(float));

    int counter = 0;
    for (size_t i = 0; i < num_elements; i++) {
        if (counter < group_size) {
            actual_y[i] = x[i % (group_size * 2)];
            predict_y[i] = y[i % (group_size * 2)];
        } else if (counter < 2 * group_size) {
            actual_y[i] = x[i % (group_size * 2) + group_size];
            predict_y[i] = y[i % (group_size * 2) + group_size];
        }
        counter++;
        if (counter >= group_size * 2) {
            counter = 0;
        }
    }

    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads - 1) / numThreads;

    printf("*** rSquare function ***\n");
    printf("numElements = %lu\n", ARRAY_SIZE);
    printf("numBlocks = %lu, numThreads = %lu \n", numBlocks, numThreads);

    *r_square = 0;
    for (size_t i = 0; i < loope; i++) {
        // SHARED MEMORY SIZE IS 5 * SIZEOF(FLOAT) FOR THE FIVE VARIABLES
        rSquare<<<numBlocks, numThreads, 5 * sizeof(float)>>>(ARRAY_SIZE, r_square, predict_y, actual_y);
        cudaDeviceSynchronize();
    }

    printf("R^2: %f\n", *r_square);

    cudaFree(actual_y);
    cudaFree(predict_y);
    cudaFree(r_square);

    return 0;
}


Overwriting CUDA_movement.cu


In [ ]:
%%shell
nvcc CUDA_movement.cu -o CUDA_movement
nvprof ./CUDA_movement

CUDA_movement.cu(55): warning #177-D: variable "ARRAY_BYTES" was declared but never referenced
      const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
                   ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

CUDA_movement.cu(82): warning #177-D: variable "z" was declared but never referenced
      float z[] = {
            ^

Enter the number of sets to group together: 1
==43338== NVPROF is profiling process 43338, command: ./CUDA_movement
*** rSquare function ***
numElements = 60
numBlocks = 1, numThreads = 1024 
R^2: 0.434535
==43338== Profiling application: ./CUDA_movement
==43338== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  393.15us         1  393.15us  393.15us  393.15us  rSquare(unsigned long, float*, float*, float*)
      API calls:   99.13%  114.68ms         3  38.227ms  9.5130us  114.65ms  cudaMallocManaged
                    0.34%  397.22us    

# **CUDA - Prefetch Memory**

In [ ]:
%%writefile CUDA_movement.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

__global__
void rSquare(size_t n, float *r_square, float *predict_y, float *actual_y) {
    extern __shared__ float shared_mem[];
    float *sum_x = &shared_mem[0];
    float *sum_y = &shared_mem[1];
    float *sum_xy = &shared_mem[2];
    float *sum_x2 = &shared_mem[3];
    float *sum_y2 = &shared_mem[4];

    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    // INITIALIZE SHARED MEMORY VALUES
    if (threadIdx.x == 0) {
        *sum_x = 0;
        *sum_y = 0;
        *sum_xy = 0;
        *sum_x2 = 0;
        *sum_y2 = 0;
    }

    __syncthreads();

    // CALCULATE SUMS IN PARALLEL
    for (int i = index; i < n; i += stride) {
        atomicAdd(sum_x, predict_y[i]);
        atomicAdd(sum_y, actual_y[i]);
        atomicAdd(sum_xy, predict_y[i] * actual_y[i]);
        atomicAdd(sum_x2, predict_y[i] * predict_y[i]);
        atomicAdd(sum_y2, actual_y[i] * actual_y[i]);
    }

    __syncthreads();

    // COMPUTE THE RESULT IN THE FIRST THREAD OF EACH BLOCK
    if (threadIdx.x == 0) {
        float numerator = (n * (*sum_xy)) - (*sum_x * *sum_y);
        float denominator = sqrtf((n * (*sum_x2) - (*sum_x) * (*sum_x)) *
                                   (n * (*sum_y2) - (*sum_y) * (*sum_y)));

        if (denominator != 0) {
            *r_square = powf(numerator / denominator, 2);
        } else {
            *r_square = 0;
        }
    }
}

int main() {
    const size_t ARRAY_SIZE = 3;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    const size_t loope = 1;

    // SAMPLE DATA
    float data_x[ARRAY_SIZE] = {1.2, 1, 2};
    float data_y[ARRAY_SIZE] = {0, 5, 2};

    float *actual_y, *predict_y, *r_square;
    cudaMallocManaged(&actual_y, ARRAY_BYTES);
    cudaMallocManaged(&predict_y, ARRAY_BYTES);
    cudaMallocManaged(&r_square, sizeof(float));

    int device;
    cudaGetDevice(&device);
    cudaMemAdvise(actual_y, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(predict_y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
    cudaMemPrefetchAsync(r_square, ARRAY_BYTES, device, NULL);

    for (size_t i = 0; i < ARRAY_SIZE; i++) {
        actual_y[i] = data_x[i];
        predict_y[i] = data_y[i];
    }

    cudaMemPrefetchAsync(actual_y, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(predict_y, ARRAY_BYTES, device, NULL);

    size_t numThreads = 256;
    size_t numBlocks = (ARRAY_SIZE + numThreads - 1) / numThreads;

    printf("*** rSquare function ***\n");
    printf("numElements = %lu\n", ARRAY_SIZE);
    printf("numBlocks = %lu, numThreads = %lu \n", numBlocks, numThreads);

    *r_square = 0;
    for (size_t i = 0; i < loope; i++) {
        // SHARED MEMORY SIZE IS 5 * SIZEOF(FLOAT) FOR THE FIVE VARIABLES
        rSquare<<<numBlocks, numThreads, 5 * sizeof(float)>>>(ARRAY_SIZE, r_square, predict_y, actual_y);
        cudaDeviceSynchronize();
    }

    printf("R^2: %f\n", *r_square);

    cudaFree(actual_y);
    cudaFree(predict_y);
    cudaFree(r_square);

    return 0;
}


Overwriting CUDA_movement.cu


In [ ]:
%%shell
nvcc CUDA_movement.cu -o CUDA_movement
nvprof ./CUDA_movement

==25403== NVPROF is profiling process 25403, command: ./CUDA_movement
*** rSquare function ***
numElements = 4
numBlocks = 1, numThreads = 256 
R^2: 0.311988
==25403== Profiling application: ./CUDA_movement
==25403== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  145.25us         1  145.25us  145.25us  145.25us  rSquare(unsigned long, float*, float*, float*)
      API calls:   99.13%  116.00ms         3  38.665ms  5.7480us  115.96ms  cudaMallocManaged
                    0.23%  274.00us         3  91.332us  8.0120us  254.71us  cudaMemPrefetchAsync
                    0.22%  261.11us         1  261.11us  261.11us  261.11us  cudaLaunchKernel
                    0.13%  150.55us         3  50.182us  8.6880us  105.46us  cudaFree
                    0.13%  150.26us         1  150.26us  150.26us  150.26us  cudaDeviceSynchronize
                    0.12%  134.70us       114  1.1810us     135ns  56.126us  cuDeviceGe

# **Results**